#  Standard Feldman Cousins Demo
Standard demo of the Feldman-Cousins calculator
StandardFeldmanCousinsDemo

This is a standard demo that can be used with any ROOT file
prepared in the standard way.  You specify:
 - name for input ROOT file
 - name of workspace inside ROOT file that holds model and data
 - name of ModelConfig that specifies details for calculator tools
 - name of dataset

With default parameters the macro will attempt to run the
standard hist2workspace example and read the ROOT file
that it produces.

The actual heart of the demo is only about 10 lines long.

The FeldmanCousins tools is a classical frequentist calculation
based on the Neyman Construction.  The test statistic can be
generalized for nuisance parameters by using the profile likelihood ratio.
But unlike the ProfileLikelihoodCalculator, this tool explicitly
builds the sampling distribution of the test statistic via toy Monte Carlo.




**Author:** Kyle Cranmer  
<i><small>This notebook tutorial was automatically generated with <a href= "https://github.com/root-project/root/blob/master/documentation/doxygen/converttonotebook.py">ROOTBOOK-izer</a> from the macro found in the ROOT repository  on Thursday, August 29, 2019 at 03:27 AM.</small></i>

In [1]:
%%cpp -d
#include "TFile.h"
#include "TROOT.h"
#include "TH1F.h"
#include "TSystem.h"

#include "RooWorkspace.h"
#include "RooAbsData.h"

#include "RooStats/ModelConfig.h"
#include "RooStats/FeldmanCousins.h"
#include "RooStats/ToyMCSampler.h"
#include "RooStats/PointSetInterval.h"
#include "RooStats/ConfidenceBelt.h"

In [2]:
%%cpp -d
// This is a workaround to make sure the namespace is used inside functions
using namespace RooFit;
using namespace RooStats;

 Arguments are defined. 

In [3]:
const char *infile = "";
const char *workspaceName = "combined";
const char *modelConfigName = "ModelConfig";
const char *dataName = "obsData";

-------------------------------------------------------
 First part is just to access a user-defined file
 or create the standard example file if it doesn't exist

In [4]:
const char *filename = "";
if (!strcmp(infile, "")) {
   filename = "results/example_combined_GaussExample_model.root";
   bool fileExist = !gSystem->AccessPathName(filename); // note opposite return code
   // if file does not exists generate with histfactory
   if (!fileExist) {
#ifdef _WIN32
      cout << "HistFactory file cannot be generated on Windows - exit" << endl;
      return;
#endif
      // Normally this would be run on the command line
      cout << "will run standard hist2workspace example" << endl;
      gROOT->ProcessLine(".! prepareHistFactory .");
      gROOT->ProcessLine(".! hist2workspace config/example.xml");
      cout << "\n\n---------------------" << endl;
      cout << "Done creating example input" << endl;
      cout << "---------------------\n\n" << endl;
   }

} else
   filename = infile;

Try to open the file

In [5]:
TFile *file = TFile::Open(filename);


RooFit v3.60 -- Developed by Wouter Verkerke and David Kirkby 
                Copyright (C) 2000-2013 NIKHEF, University of California & Stanford University
                All rights reserved, please read http://roofit.sourceforge.net/license.txt



If input file was specified byt not found, quit

In [6]:
if (!file) {
   cout << "StandardRooStatsDemoMacro: Input file " << filename << " is not found" << endl;
   return;
}

-------------------------------------------------------
 Tutorial starts here
 -------------------------------------------------------

Get the workspace out of the file

In [7]:
RooWorkspace *w = (RooWorkspace *)file->Get(workspaceName);
if (!w) {
   cout << "workspace not found" << endl;
   return;
}

Get the modelconfig out of the file

In [8]:
ModelConfig *mc = (ModelConfig *)w->obj(modelConfigName);

Get the modelconfig out of the file

In [9]:
RooAbsData *data = w->data(dataName);

Make sure ingredients are found

In [10]:
if (!data || !mc) {
   w->Print();
   cout << "data or ModelConfig was not found" << endl;
   return;
}

-------------------------------------------------------
 create and use the FeldmanCousins tool
 to find and plot the 95% confidence interval
 on the parameter of interest as specified
 in the model config

In [11]:
FeldmanCousins fc(*data, *mc);
fc.SetConfidenceLevel(0.95); // 95% interval

Fc.additionalntoysfactor(0.1); // to speed up the result

In [12]:
fc.UseAdaptiveSampling(true); // speed it up a bit
fc.SetNBins(10);              // set how many points per parameter of interest to scan
fc.CreateConfBelt(true);      // save the information in the belt for plotting

Since this tool needs to throw toy mc the pdf needs to be
 extended or the tool needs to know how many entries in a dataset
 per pseudo experiment.
 In the 'number counting form' where the entries in the dataset
 are counts, and not values of discriminating variables, the
 datasets typically only have one entry and the PDF is not
 extended.

In [13]:
if (!mc->GetPdf()->canBeExtended()) {
   if (data->numEntries() == 1)
      fc.FluctuateNumDataEntries(false);
   else
      cout << "Not sure what to do about this model" << endl;
}

We can use proof to speed things along in parallel
  ProofConfig pc(*w, 1, "workers=4", kFALSE);
  ToyMCSampler*  toymcsampler = (ToyMCSampler*) fc.GetTestStatSampler();
  toymcsampler->SetProofConfig(&pc); // enable proof

Now get the interval

In [14]:
PointSetInterval *interval = fc.GetInterval();
ConfidenceBelt *belt = fc.GetConfidenceBelt();


=== Using the following for ModelConfig ===
Observables:             RooArgSet:: = (obs_x_channel1,weightVar,channelCat)
Parameters of Interest:  RooArgSet:: = (SigXsecOverSM)
Nuisance Parameters:     RooArgSet:: = (alpha_syst2,alpha_syst3,gamma_stat_channel1_bin_0,gamma_stat_channel1_bin_1)
Global Observables:      RooArgSet:: = (nom_alpha_syst2,nom_alpha_syst3,nom_gamma_stat_channel1_bin_0,nom_gamma_stat_channel1_bin_1)
PDF:                     RooSimultaneous::simPdf[ indexCat=channelCat channel1=model_channel1 ] = 0.174888

FeldmanCousins: ntoys per point: adaptive
FeldmanCousins: nEvents per toy will fluctuate about  expectation
FeldmanCousins: Model has nuisance parameters, will do profile construction
FeldmanCousins: # points to test = 10
lookup index = 0
NeymanConstruction: Prog: 1/10 total MC = 78 this test stat = 1.323
 SigXsecOverSM=0.15 alpha_syst2=0.602204 alpha_syst3=0.227781 gamma_stat_channel1_bin_0=1.03121 gamma_stat_channel1_bin_1=1.04626 [-1e+30, 3.00321]  in interv

Print out the interval on the first parameter of interest

In [15]:
RooRealVar *firstPOI = (RooRealVar *)mc->GetParametersOfInterest()->first();
cout << "\n95% interval on " << firstPOI->GetName() << " is : [" << interval->LowerLimit(*firstPOI) << ", "
     << interval->UpperLimit(*firstPOI) << "] " << endl;


95% interval on SigXsecOverSM is : [0.15, 2.25] 


---------------------------------------------
 No nice plots yet, so plot the belt by hand

Ask the calculator which points were scanned

In [16]:
RooDataSet *parameterScan = (RooDataSet *)fc.GetPointsToScan();
RooArgSet *tmpPoint;

Make a histogram of parameter vs. threshold

In [17]:
TH1F *histOfThresholds =
   new TH1F("histOfThresholds", "", parameterScan->numEntries(), firstPOI->getMin(), firstPOI->getMax());

Loop through the points that were tested and ask confidence belt
 what the upper/lower thresholds were.
 For FeldmanCousins, the lower cut off is always 0

In [18]:
for (Int_t i = 0; i < parameterScan->numEntries(); ++i) {
   tmpPoint = (RooArgSet *)parameterScan->get(i)->clone("temp");
   double arMax = belt->GetAcceptanceRegionMax(*tmpPoint);
   double arMin = belt->GetAcceptanceRegionMax(*tmpPoint);
   double poiVal = tmpPoint->getRealValue(firstPOI->GetName());
   histOfThresholds->Fill(poiVal, arMax);
}
histOfThresholds->SetMinimum(0);
histOfThresholds->Draw();

Info in <TCanvas::MakeDefCanvas>:  created default TCanvas with name c1


Draw all canvases 

In [19]:
%jsroot on
gROOT->GetListOfCanvases()->Draw()